In [3]:
import sys
import os

project_root = os.path.abspath("..")
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [4]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from data.data_loader import generateGauss
from utils.integral import grid
from Models.classification import SVM
from utils.kernel import Kernel

In [5]:
h = 0.01
x_grid = grid(h, start=-2, end=12)

mu_A = [1, 1.5, 2, 2.5, 6]
sig_A = 0.4 * np.ones_like(mu_A)
mu_B = [4.5, 7.5, 8.5, 8, 8.5]
sig_B = 0.9 * np.ones_like(mu_A)

class1_data = generateGauss(mu_A, sig_A, x_grid)
class2_data = generateGauss(mu_B, sig_B, x_grid)
all_pdfs = np.vstack([class1_data, class2_data])

y = np.array([0]*len(class1_data) + [1]*len(class2_data))
n_A, n_B = len(class1_data), len(class2_data)

In [6]:
print("\n=== SVM ===")

# Tạo kernel cho training
kc = Kernel(h=0.01, Dim=1)
K_train = kc.L1(all_pdfs)  # (n_train, n_train)

# Train SVM với precomputed kernel
svm_model = SVM.Model(C=1)
svm_model.fit(K_train, y)

# Predict trên training set
preds_svm = svm_model.predict(K_train)
acc_svm = np.mean(preds_svm == y)
print(f"[SVM] Accuracy: {acc_svm * 100:.2f}%")

# Lấy probability (class 1) trên training set
probas_svm = svm_model.predict_proba(K_train)[:, 1]  # xác suất lớp 1
print(probas_svm)


=== SVM ===
[SVM] Accuracy: 100.00%
[0.27029599 0.2491102  0.2509187  0.27693571 0.34049642 0.60447121
 0.71074397 0.76537283 0.75457029 0.76537283]


In [7]:
new_pdf = generateGauss([3], [0.9], x_grid).ravel()

# Tạo kernel giữa (train, new_sample) → (n_train, 1)
K_full = kc.L1(np.vstack([all_pdfs, new_pdf[None, :]]))  # (n_train+1, n_train+1)
K_new = K_full[:-1, -1][:, None]  # (n_train, 1)

# Predict class và probability cho new_pdf
pred_svm = svm_model.predict(K_new)[0]
prob_svm = svm_model.predict_proba(K_new)[0, 1]
print(f"[SVM] New sample → Class: {pred_svm}, Probability (class 1): {prob_svm:.4f}")

[SVM] New sample → Class: 0, Probability (class 1): 0.3940
